In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
import gensim
import csv
import json
import nltk
from gensim import corpora, models
import numpy as np
from nltk.corpus import wordnet
from sklearn.neighbors import NearestNeighbors

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36"}

In [ ]:
moviedf = pd.DataFrame(columns = ['movie', 'summary'])

In [8]:
#source = requests.get('https://www.imdb.com/search/title/?title_type=movie&explore=title_type,genres', headers = headers).text
source = requests.get('https://www.imdb.com/search/title/?title_type=movie&start=2001&explore=title_type,genres&ref_=adv_nxt', headers = headers).text

soup = BeautifulSoup(source, 'html.parser')

count = 0
pages = 20 #set amount of pages to browse for movies
current_page = soup
for page in range(pages):
    blocks = current_page.find('div', class_ = 'article').find('div', class_ = 'lister-list').find_all('div', class_ = 'lister-item mode-advanced')
    for block in blocks:
        count += 1
        print('pages', page+1, 'count', count)
        #if block.find('div', class_ = 'inline-block ratings-imdb-rating').find('strong') is None: # don't care about score for now
        #    continue
        movie = block.find('h3', class_ = 'lister-item-header').a.text
        movie_code = block.find('div', class_ = 'ribbonize').attrs['data-tconst']
        summary_page = 'https://www.imdb.com/title/' + movie_code + '/plotsummary'
        source = requests.get(summary_page, headers=headers).text
        soup = BeautifulSoup(source, 'html.parser')
        is_summary_not_exist = soup.find('li', class_ = 'ipl-zebra-list__item', id = 'no-summary-content')
        if is_summary_not_exist is not None: # triple negative here -> check if not exist then skip
            continue
        summary_blocks = soup.find('ul', class_ = 'ipl-zebra-list', id = 'plot-summaries-content').find_all('li', class_ = 'ipl-zebra-list__item')
        summary = ''
        for summary_block in summary_blocks:
            summary += summary_block.p.text
        moviedf.loc[len(moviedf)] = [movie, summary]
    next_page_button = current_page.find('a', class_ = 'lister-page-next next-page').attrs['href']
    next_page_url = 'https://www.imdb.com' + next_page_button
    source = requests.get(next_page_url, headers = headers).text
    soup = BeautifulSoup(source, 'html.parser')
    current_page = soup
    

pages 1 count 1
pages 1 count 2
pages 1 count 3
pages 1 count 4
pages 1 count 5
pages 1 count 6
pages 1 count 7
pages 1 count 8
pages 1 count 9
pages 1 count 10
pages 1 count 11
pages 1 count 12
pages 1 count 13
pages 1 count 14
pages 1 count 15
pages 1 count 16
pages 1 count 17
pages 1 count 18
pages 1 count 19
pages 1 count 20
pages 1 count 21
pages 1 count 22
pages 1 count 23
pages 1 count 24
pages 1 count 25
pages 1 count 26
pages 1 count 27
pages 1 count 28
pages 1 count 29
pages 1 count 30
pages 1 count 31
pages 1 count 32
pages 1 count 33
pages 1 count 34
pages 1 count 35
pages 1 count 36
pages 1 count 37
pages 1 count 38
pages 1 count 39
pages 1 count 40
pages 1 count 41
pages 1 count 42
pages 1 count 43
pages 1 count 44
pages 1 count 45
pages 1 count 46
pages 1 count 47
pages 1 count 48
pages 1 count 49
pages 1 count 50
pages 2 count 51
pages 2 count 52
pages 2 count 53
pages 2 count 54
pages 2 count 55
pages 2 count 56
pages 2 count 57
pages 2 count 58
pages 2 count 59
pages 

pages 10 count 462
pages 10 count 463
pages 10 count 464
pages 10 count 465
pages 10 count 466
pages 10 count 467
pages 10 count 468
pages 10 count 469
pages 10 count 470
pages 10 count 471
pages 10 count 472
pages 10 count 473
pages 10 count 474
pages 10 count 475
pages 10 count 476
pages 10 count 477
pages 10 count 478
pages 10 count 479
pages 10 count 480
pages 10 count 481
pages 10 count 482
pages 10 count 483
pages 10 count 484
pages 10 count 485
pages 10 count 486
pages 10 count 487
pages 10 count 488
pages 10 count 489
pages 10 count 490
pages 10 count 491
pages 10 count 492
pages 10 count 493
pages 10 count 494
pages 10 count 495
pages 10 count 496
pages 10 count 497
pages 10 count 498
pages 10 count 499
pages 10 count 500
pages 11 count 501
pages 11 count 502
pages 11 count 503
pages 11 count 504
pages 11 count 505
pages 11 count 506
pages 11 count 507
pages 11 count 508
pages 11 count 509
pages 11 count 510
pages 11 count 511
pages 11 count 512
pages 11 count 513
pages 11 cou

pages 18 count 894
pages 18 count 895
pages 18 count 896
pages 18 count 897
pages 18 count 898
pages 18 count 899
pages 18 count 900
pages 19 count 901
pages 19 count 902
pages 19 count 903
pages 19 count 904
pages 19 count 905
pages 19 count 906
pages 19 count 907
pages 19 count 908
pages 19 count 909
pages 19 count 910
pages 19 count 911
pages 19 count 912
pages 19 count 913
pages 19 count 914
pages 19 count 915
pages 19 count 916
pages 19 count 917
pages 19 count 918
pages 19 count 919
pages 19 count 920
pages 19 count 921
pages 19 count 922
pages 19 count 923
pages 19 count 924
pages 19 count 925
pages 19 count 926
pages 19 count 927
pages 19 count 928
pages 19 count 929
pages 19 count 930
pages 19 count 931
pages 19 count 932
pages 19 count 933
pages 19 count 934
pages 19 count 935
pages 19 count 936
pages 19 count 937
pages 19 count 938
pages 19 count 939
pages 19 count 940
pages 19 count 941
pages 19 count 942
pages 19 count 943
pages 19 count 944
pages 19 count 945
pages 19 cou

In [9]:
moviedf.to_csv('summary.csv', index = False)

In [3]:
moviedf = pd.read_csv('summary.csv')
summary_text = moviedf['summary']
moviedf.head()

,movie,summary
0,El hoyo,A vertical prison with one cell per level. Two...
1,The Gentlemen,An American expat tries to sell off his highly...
2,Once Upon a Time... in Hollywood,A faded television actor and his stunt double ...
3,Contagion,"Healthcare professionals, government officials..."
4,Onward,Two elven brothers embark on a quest to bring ...


In [4]:
summary_text = summary_text.apply(lambda x: x.strip())
summary_text = summary_text.apply(lambda x: x.lower())
summary_text = summary_text.apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))
summary_text = summary_text.apply(lambda x: word_tokenize(x))
summary_text = summary_text.apply(lambda x: [i for i in x if len(i) >= 3])
summary_text = summary_text.apply(lambda x: [i for i in x if i not in stopwords.words("english")])
lemmatizer = WordNetLemmatizer()
summary_text = summary_text.apply(lambda x: [lemmatizer.lemmatize(i, pos="v") for i in x])

In [5]:
summary_dictionary = gensim.corpora.Dictionary(summary_text)
print('Total documents:', summary_dictionary.num_docs)
print('Total words:', summary_dictionary.num_pos)
print('Distinct words:', len(summary_dictionary))

Total documents: 2999
Total words: 598500
Distinct words: 29573


In [6]:
summary_dictionary.filter_extremes(no_below = 30, no_above=0.3)
print('Total documents:', summary_dictionary.num_docs)
print('Total words:', summary_dictionary.num_pos)
print('Distinct words:', len(summary_dictionary))

Total documents: 2999
Total words: 598500
Distinct words: 2281


In [7]:
summary_bow_corpus = [summary_dictionary.doc2bow(doc) for doc in summary_text]

In [8]:
summary_lda_model = gensim.models.LdaMulticore(summary_bow_corpus, 
                                   num_topics = 50,
                                   id2word = summary_dictionary,                                    
                                   passes = 30,
                                   workers = 4,
                                   alpha = 0.02,
                                   eta = 0.1
                                   
                                   )
for idx, topic in summary_lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.040*"war" + 0.012*"american" + 0.011*"soldier" + 0.010*"men" + 0.009*"british" + 0.009*"first" + 0.008*"country" + 0.008*"civil" + 0.008*"fight" + 0.008*"french"
Topic: 1 
Words: 0.068*"frank" + 0.055*"jones" + 0.016*"texas" + 0.015*"hospital" + 0.013*"doctor" + 0.009*"escape" + 0.008*"money" + 0.007*"father" + 0.007*"stone" + 0.006*"horse"
Topic: 2 
Words: 0.032*"city" + 0.017*"virus" + 0.014*"people" + 0.013*"survivors" + 0.010*"alice" + 0.009*"wall" + 0.009*"york" + 0.008*"escape" + 0.008*"group" + 0.007*"survive"
Topic: 3 
Words: 0.044*"bond" + 0.038*"jam" + 0.015*"agent" + 0.012*"steal" + 0.011*"british" + 0.010*"kill" + 0.010*"plan" + 0.008*"send" + 0.008*"sir" + 0.007*"plot"
Topic: 4 
Words: 0.037*"anna" + 0.033*"nick" + 0.033*"johnny" + 0.015*"boss" + 0.008*"martin" + 0.008*"first" + 0.008*"village" + 0.006*"job" + 0.006*"hire" + 0.006*"want"
Topic: 5 
Words: 0.078*"rocky" + 0.024*"fight" + 0.020*"champion" + 0.017*"train" + 0.017*"river" + 0.016*"box" + 0.01

In [23]:
summary_tfidf = models.TfidfModel(summary_bow_corpus)
summary_corpus_tfidf = summary_tfidf[summary_bow_corpus]

In [24]:
summary_tfidf_lda_model =  gensim.models.LdaMulticore(summary_corpus_tfidf, 
                                   num_topics = 50,
                                   id2word = summary_dictionary,                                    
                                   passes = 30,
                                   workers = 4,
                                   alpha = 0.02,
                                   eta = 0.1
                                   )
for idx, topic in summary_tfidf_lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.000*"louis" + 0.000*"ryan" + 0.000*"beast" + 0.000*"france" + 0.000*"school" + 0.000*"rachel" + 0.000*"pet" + 0.000*"van" + 0.000*"girls" + 0.000*"lady"
Topic: 1 
Words: 0.003*"focus" + 0.003*"master" + 0.002*"movie" + 0.002*"base" + 0.001*"joe" + 0.001*"center" + 0.001*"character" + 0.001*"eye" + 0.001*"around" + 0.000*"writer"
Topic: 2 
Words: 0.004*"early" + 0.003*"complicate" + 0.002*"student" + 0.002*"hearts" + 0.002*"1970s" + 0.002*"residents" + 0.002*"serial" + 0.002*"involve" + 0.002*"film" + 0.002*"strike"
Topic: 3 
Words: 0.000*"romantic" + 0.000*"tale" + 0.000*"france" + 0.000*"russia" + 0.000*"network" + 0.000*"china" + 0.000*"jack" + 0.000*"match" + 0.000*"soviet" + 0.000*"military"
Topic: 4 
Words: 0.000*"award" + 0.000*"wreck" + 0.000*"dance" + 0.000*"judge" + 0.000*"south" + 0.000*"action" + 0.000*"epic" + 0.000*"william" + 0.000*"island" + 0.000*"thriller"
Topic: 5 
Words: 0.000*"mysteriously" + 0.000*"stake" + 0.000*"nick" + 0.000*"multiple" + 0.000

In [67]:
stopwords.words("english")

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [3]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('celebrating' , pos='v')

'celebrate'

In [53]:
for idx, topic in summary_lda_model.print_topics():
    print(topic)

0.034*"los" + 0.033*"angeles" + 0.030*"film" + 0.021*"dog" + 0.019*"star" + 0.018*"hollywood" + 0.016*"car" + 0.015*"movie" + 0.013*"actor" + 0.009*"director"
0.044*"book" + 0.015*"mary" + 0.011*"movie" + 0.010*"treasure" + 0.010*"john" + 0.009*"sister" + 0.008*"eddie" + 0.007*"sisters" + 0.007*"ben" + 0.006*"writer"
0.017*"ben" + 0.012*"johnny" + 0.012*"music" + 0.012*"san" + 0.012*"band" + 0.011*"job" + 0.009*"chris" + 0.009*"francisco" + 0.008*"money" + 0.008*"play"
0.053*"town" + 0.022*"small" + 0.014*"race" + 0.012*"news" + 0.011*"dream" + 0.009*"couple" + 0.009*"local" + 0.008*"group" + 0.007*"people" + 0.007*"big"
0.020*"human" + 0.014*"strange" + 0.013*"train" + 0.012*"race" + 0.012*"save" + 0.010*"protect" + 0.009*"monsters" + 0.009*"ancient" + 0.009*"future" + 0.008*"discover"
0.022*"baby" + 0.016*"god" + 0.013*"child" + 0.011*"kill" + 0.011*"death" + 0.010*"evil" + 0.008*"leave" + 0.008*"force" + 0.008*"name" + 0.008*"son"
0.020*"danny" + 0.013*"ryan" + 0.013*"rachel" + 0.01

In [13]:
result_list = np.array(summary_lda_model.inference(summary_bow_corpus)[0])

In [10]:
summary_lda_model.inference(summary_bow_corpus)[0]

array([[0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       ...,
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02]], dtype=float32)

In [11]:
summary_lda_model.inference(summary_bow_corpus)

(array([[0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
        [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
        [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
        ...,
        [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
        [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
        [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02]], dtype=float32), None)

In [16]:
print(result_list.shape)

(2999, 50)


In [17]:
top_topics = summary_lda_model.top_topics(summary_bow_corpus)

In [19]:
avg_topic_coherence = sum([t[1] for t in top_topics]) / 50
print('Average topic coherence: %.4f.' % avg_topic_coherence)

Average topic coherence: -2.5180.


In [20]:
top_topics

[([(0.02371109, 'love'),
   (0.010112767, 'woman'),
   (0.010045391, 'relationship'),
   (0.009657456, 'family'),
   (0.009057429, 'marry'),
   (0.0075609605, 'fall'),
   (0.0066817296, 'want'),
   (0.0063692937, 'years'),
   (0.0061184056, 'husband'),
   (0.0059276912, 'day'),
   (0.005893832, 'feel'),
   (0.005592933, 'see'),
   (0.0053351587, 'old'),
   (0.0053202407, 'mother'),
   (0.0052952864, 'together'),
   (0.005245967, 'also'),
   (0.005177717, 'wife'),
   (0.00492008, 'marriage'),
   (0.0049022827, 'may'),
   (0.0048793717, 'move')],
  -1.3637128789857649),
 ([(0.041784454, 'school'),
   (0.02278466, 'high'),
   (0.022215474, 'friends'),
   (0.017881094, 'girl'),
   (0.011910467, 'friend'),
   (0.010848556, 'best'),
   (0.010467736, 'boy'),
   (0.0100209005, 'college'),
   (0.009498268, 'day'),
   (0.009333334, 'party'),
   (0.008334779, 'want'),
   (0.006951923, 'kid'),
   (0.006831142, 'student'),
   (0.0067841723, 'year'),
   (0.006235066, 'girls'),
   (0.0061897645, 'lov

In [21]:
summary_lda_model.save('summary_lda_model.model')

In [41]:
summary_lda_model.print_topic(48, topn = 30)

'0.016*"treasure" + 0.014*"arthur" + 0.013*"father" + 0.010*"lose" + 0.010*"search" + 0.010*"adventure" + 0.010*"years" + 0.009*"son" + 0.009*"journey" + 0.009*"discover" + 0.008*"fish" + 0.008*"back" + 0.008*"evil" + 0.008*"clue" + 0.007*"save" + 0.007*"bring" + 0.007*"embark" + 0.006*"home" + 0.006*"power" + 0.006*"name" + 0.006*"set" + 0.006*"across" + 0.006*"return" + 0.006*"knight" + 0.005*"video" + 0.005*"island" + 0.005*"reporter" + 0.005*"ben" + 0.005*"devil" + 0.005*"hide"'

In [27]:
summary_lda_model.print_topic(40, topn = 30)

'0.014*"joe" + 0.010*"escape" + 0.010*"prison" + 0.009*"way" + 0.009*"car" + 0.008*"driver" + 0.008*"name" + 0.008*"back" + 0.007*"run" + 0.006*"money" + 0.006*"island" + 0.006*"night" + 0.006*"set" + 0.006*"rescue" + 0.006*"job" + 0.005*"years" + 0.005*"end" + 0.005*"break" + 0.004*"need" + 0.004*"drive" + 0.004*"plan" + 0.004*"survive" + 0.004*"little" + 0.004*"desert" + 0.004*"head" + 0.004*"road" + 0.004*"return" + 0.004*"day" + 0.004*"learn" + 0.004*"food"'

In [36]:
summary_lda_model.get_document_topics(summary_bow_corpus[100], minimum_probability=0.0)

[(0, 0.00016003897),
 (1, 0.00016003897),
 (2, 0.00016003897),
 (3, 0.00016003897),
 (4, 0.00016003897),
 (5, 0.00016003897),
 (6, 0.05875984),
 (7, 0.19466804),
 (8, 0.14320675),
 (9, 0.00016003897),
 (10, 0.00016003897),
 (11, 0.00016003897),
 (12, 0.00016003897),
 (13, 0.00016003897),
 (14, 0.00016003897),
 (15, 0.00016003897),
 (16, 0.00016003897),
 (17, 0.00016003897),
 (18, 0.011527722),
 (19, 0.00016003897),
 (20, 0.00016003897),
 (21, 0.13528238),
 (22, 0.00016003897),
 (23, 0.00016003897),
 (24, 0.00016003897),
 (25, 0.00016003897),
 (26, 0.00016003897),
 (27, 0.13139202),
 (28, 0.00016003897),
 (29, 0.00016003897),
 (30, 0.00016003897),
 (31, 0.00016003897),
 (32, 0.00016003897),
 (33, 0.00016003897),
 (34, 0.00016003897),
 (35, 0.00016003897),
 (36, 0.00016003897),
 (37, 0.00016003897),
 (38, 0.00016003897),
 (39, 0.00016003897),
 (40, 0.15268044),
 (41, 0.00016003897),
 (42, 0.00016003897),
 (43, 0.00016003897),
 (44, 0.00016003897),
 (45, 0.00016003897),
 (46, 0.0001600389

In [44]:
moviedf['summary'][100]

'A struggling musician realizes he\'s the only person on Earth who can remember The Beatles after waking up in an alternate timeline where they never existed.In Lowestoft UK, Jack Malik is a frustrated musician whose musical career is going nowhere despite the faith that his friend/manager Ellie Appleton has in him. However, on the night Jack decides to give up, the whole world is momentarily hit with a massive blackout during which Jack is hit by a bus. Upon regaining consciousness, Jack learns to his astonishment that he is apparently now the only one who knows the music of the Beatles. Realizing this improbable opportunity, Jack begins playing the music of one of the world\'s greatest rock bands, claiming it as his own. It pays off quickly and Jack becomes a worldwide musical sensation. However, Jack finds himself drifting away from Ellie, only realizing his love for her when she has become intimidated by his success, which depends on a blatant plagiarization that no one could find 

In [45]:
top_topics = summary_lda_model.get_document_topics(summary_bow_corpus[100], minimum_probability=0.0)
topic_vec = [top_topics[i][1] for i in range(20)]

In [54]:
lda_data = []
for i in range(len(moviedf)):
    top_topics = summary_lda_model.get_document_topics(summary_bow_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(50)]
    lda_data.append(topic_vec)

In [48]:
len(moviedf)

2999

In [55]:
ldadf = pd.DataFrame(lda_data)

In [76]:
knn = NearestNeighbors(n_neighbors = 30).fit(ldadf)

In [77]:
distances, indices = knn.kneighbors(ldadf)

In [66]:
moviedf['movie']

0                                                 El hoyo
1                                           The Gentlemen
2                        Once Upon a Time... in Hollywood
3                                               Contagion
4                                                  Onward
5                                       Bad Boys for Life
6                                       The Invisible Man
7                                              Underwater
8                                              Knives Out
9                                               Bloodshot
10                                                   1917
11                                           Gisaengchung
12                                               The Hunt
13      Birds of Prey: And the Fantabulous Emancipatio...
14          Star Wars: Episode IX - The Rise of Skywalker
15                                               Vivarium
16                                Jumanji: The Next Level
17            

In [84]:
movie_id = moviedf['movie'].tolist().index('Scooby-Doo 2: Monsters Unleashed')
for i in range(indices.shape[1]):
    print(i, moviedf['movie'][indices[movie_id, i]], distances[movie_id, i])

0 Scooby-Doo 2: Monsters Unleashed 0.0
1 Casper 0.3470804003375035
2 Heavyweights 0.35299961871376984
3 Fantasy Island 0.363536351350823
4 Ghostbusters: Afterlife 0.36628429572328125
5 Klaus 0.37156360890679774
6 Enola Holmes 0.387702189350282
7 Run Sweetheart Run 0.39881147574733505
8 The Woman in the Window 0.40153754583557894
9 Under the Skin 0.4071307443725266
10 What Lies Beneath 0.40820403282475654
11 The Addams Family 0.41210502873526567
12 I See You 0.41240683363562586
13 Don't Breathe 0.41839922628225523
14 Coraline 0.42926140213250974
15 Cop Out 0.4330209708917081
16 Dora and the Lost City of Gold 0.43590520598619265
17 The Intruder 0.43777237059669455
18 Honey, I Shrunk the Kids 0.4388004859600942
19 In the Tall Grass 0.4407401505230964
20 Home Alone 0.44577752518185354
21 Ready or Not 0.4486187281821684
22 Friday the 13th 0.449789280625079
23 Beetlejuice 0.4509227704217351
24 Game Night 0.45093158708285835
25 The Boy Next Door 0.4512113234530824
26 Rosemary's Baby 0.4513146

In [72]:
moviedf['summary'][moviedf['movie'].tolist().index('Joker')]

'In Gotham City, mentally troubled comedian Arthur Fleck is disregarded and mistreated by society. He then embarks on a downward spiral of revolution and bloody crime. This path brings him face-to-face with his alter-ego: the Joker.Joker centers around an origin of the iconic arch nemesis and is an original, standalone story not seen before on the big screen. Todd Phillips\' exploration of Arthur Fleck (Joaquin Phoenix), a man disregarded by society is not only a gritty character study, but also a broader cautionary tale.Arthur Fleck works as a clown and is an aspiring stand-up comic. He has mental health issues, part of which involves uncontrollable laughter. Times are tough and, due to his issues and occupation, Arthur has an even worse time than most. Over time these issues bear down on him, shaping his actions, making him ultimately take on the persona he is more known as...Joker.Arthur Fleck is a wannabe stand-up comic who suffers from many mental illnesses, including one which ca

In [74]:
moviedf['summary'][moviedf['movie'].tolist().index('Frozen II')]

'A mentally unstable veteran works as a nighttime taxi driver in New York City, where the perceived decadence and sleaze fuels his urge for violent action by attempting to liberate a presidential campaign worker and an underage prostitute.Travis Bickle is an ex-Marine and Vietnam War veteran living in New York City. As he suffers from insomnia, he spends his time working as a taxi driver at night, watching porn movies at seedy cinemas during the day, or thinking about how the world, New York in particular, has deteriorated into a cesspool. He\'s a loner who has strong opinions about what is right and wrong with mankind. For him, the one bright spot in New York humanity is Betsy, a worker on the presidential nomination campaign of Senator Charles Palantine. He becomes obsessed with her. After an incident with her, he believes he has to do whatever he needs to make the world a better place in his opinion. One of his priorities is to be the savior for Iris, a twelve-year-old runaway and p